In [1]:
import os

os.getcwd()

'e:\\Google Drive (awekim@handong.edu)\\[Project]\\[Quantum]\\02_Analysis'

In [2]:
import pandas as pd

# inst = pd.read_csv('../05_허지수박사/quant_inst_ed_eu.csv', encoding='ISO-8859-1')
inst = pd.read_csv('QuanTech_R/R file/quant_inst_ed_eu.csv', encoding='ISO-8859-1')
inst.head()

,Unnamed: 0,pubid,qc_category,institution_id_te,organization,suborganization,full_address,city,state,country,zip,pubyear,organization_clean
0,1,8064916,qc111,77541063,UNIV PATRAS,SCH NAT SCI,"UNIV PATRAS, SCH NAT SCI, DEPT MAT SCI, PATRAS...",PATRAS,NaN,GREECE,26504,2004,UNIV PATRAS
1,2,8064916,qc111,77541064,UNIV PATRAS,DEPT MAT SCI,"UNIV PATRAS, SCH NAT SCI, DEPT MAT SCI, PATRAS...",PATRAS,NaN,GREECE,26504,2004,UNIV PATRAS
2,3,8064916,qc111,84907542,UNIVERSITY OF PATRAS,SCH NAT SCI,"UNIV PATRAS, SCH NAT SCI, DEPT MAT SCI, PATRAS...",PATRAS,NaN,GREECE,26504,2004,UNIV PATRAS
3,4,8064916,qc111,84907543,UNIVERSITY OF PATRAS,DEPT MAT SCI,"UNIV PATRAS, SCH NAT SCI, DEPT MAT SCI, PATRAS...",PATRAS,NaN,GREECE,26504,2004,UNIV PATRAS
4,5,8064916,qc111,70835111,UNIV DURHAM,DEPT PHYS,"UNIV DURHAM, DEPT PHYS, DURHAM DH1 3LE, ENGLAND",DURHAM,NaN,ENGLAND,DH1 3LE,2004,UNIV DURHAM


In [3]:
# Replace abbreviations e.g., 'UNIV' with 'UNIVERSITY' and 'INST' with 'INSTITUTION' in the 'organization' column
inst['standardized_organization'] = inst['organization'].str.replace(r'\bINST\b', 'INSTITUTION', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bHOP\b', 'HOSPITAL', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bHOSP\b', 'HOSPITAL', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bTECHNOL\b', 'TECHNOLOGY', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bMED\b', 'MEDICAL', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bNATL\b', 'NATIONAL', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bCTR\b', 'CENTRE', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bNUCL\b', 'NUCLEAR', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bRES\b', 'RESEARCH', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bPUBL\b', 'PUBLIC', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bAGCY\b', 'AGENCY', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bHLTH\b', 'HEALTH', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bSERV\b', 'SERVICE', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bCOLL\b', 'COLLEGE', regex=True)
inst['standardized_organization'] = inst['organization'].str.replace(r'\bMED\b', 'MEDICAL', regex=True)

In [4]:
# Replace "UNIV" with "UNIVERSITY" and "UNIVERSITY OF" in the 'standardized_organization' column

def modify_organization_name(name):
    # Check if the value is a string
    if isinstance(name, str):
        # Check if already correctly formatted or not containing "UNIV"
        if "UNIVERSITY" in name or "UNIV" not in name:
            return name
        elif name.endswith('UNIV'):
            return name.replace('UNIV', 'UNIVERSITY')
        else:
            return name.replace('UNIV', 'UNIVERSITY OF')
    else:
        # Return the value as it is if it's not a string
        return name

inst['standardized_organization'] = inst['standardized_organization'].apply(modify_organization_name)
inst[['organization', 'standardized_organization']].head()

,organization,standardized_organization
0,UNIV PATRAS,UNIVERSITY OF PATRAS
1,UNIV PATRAS,UNIVERSITY OF PATRAS
2,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS
3,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS
4,UNIV DURHAM,UNIVERSITY OF DURHAM


In [5]:
inst.standardized_organization.value_counts()

CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS)    26858
UNIVERSITY OF CAMBRIDGE                                 6672
CONSIGLIO NAZIONALE DELLE RICERCHE (CNR)                6607
UNIVERSITY OF OXFORD                                    6594
MAX PLANCK SOCIETY                                      5215
                                                       ...  
CA FONCELLO HOSP                                           1
IRCCS BURLO GAROFOLO                                       1
INST MATERNAL & CHILD HLTH IRCCS BURLO GAROFOLO            1
CTR RIFERIMENTO ONCOL AVIANO CRO IRCCS                     1
ARQUIMEA RES CTR                                           1
Name: standardized_organization, Length: 11405, dtype: int64

In [ ]:
### import fuzzywuzzy package

# !pip install fuzzywuzzy python-Levenshtein

In [7]:
from fuzzywuzzy import fuzz
from collections import defaultdict

# This function will return True if two names are similar based on a set threshold
def are_similar(name1, name2, threshold=90):
    return fuzz.token_sort_ratio(name1, name2) >= threshold

# Create clusters of similar names
clusters = defaultdict(list)
names = inst['standardized_organization'].unique()

for name in names:
    found_cluster = False
    for cluster_name in clusters:
        if are_similar(name, cluster_name):
            clusters[cluster_name].append(name)
            found_cluster = True
            break
    if not found_cluster:
        clusters[name].append(name)

# For each cluster, select the most common name as the standard name
standard_names = {}
for cluster, names in clusters.items():
    standard_name = max(names, key=names.count)  
    for name in names:
        standard_names[name] = standard_name

# Apply the standardization
inst['final_standardized_organization'] = inst['standardized_organization'].map(standard_names)

# Check the result
print(inst[['standardized_organization', 'final_standardized_organization']])

       standardized_organization final_standardized_organization
0           UNIVERSITY OF PATRAS            UNIVERSITY OF PATRAS
1           UNIVERSITY OF PATRAS            UNIVERSITY OF PATRAS
2           UNIVERSITY OF PATRAS            UNIVERSITY OF PATRAS
3           UNIVERSITY OF PATRAS            UNIVERSITY OF PATRAS
4           UNIVERSITY OF DURHAM            UNIVERSITY OF DURHAM
...                          ...                             ...
520856     UNIVERSITY OF BRISTOL           UNIVERSITY OF BRISTOL
520857     UNIVERSITY OF BRISTOL           UNIVERSITY OF BRISTOL
520858     UNIVERSITY OF BRISTOL           UNIVERSITY OF BRISTOL
520859     UNIVERSITY OF BRISTOL           UNIVERSITY OF BRISTOL
520860     UNIVERSITY OF BRISTOL           UNIVERSITY OF BRISTOL

[520861 rows x 2 columns]


In [9]:
# Manual Edits
# if 'Source' and 'Target' columns contain 'A DUPLICATED NAME OF THE ORGANIZATION',
# then replace it with 'MERGED NAME OF THE ORGANIZATION'
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS)', 'CNRS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CZECH ACAD SCI', 'CZECH ACADEMY OF SCIENCES')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS) - INSTITUTE FOR ENGINEERING & SYSTEMS SCIENCES (INSIS)', 'CNRS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('KATHOLIEKE UNIVERSITY OF LEUVEN', 'KU LEUVEN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('POLITECN MILAN', 'POLYTECHNIC UNIVERSITY OF MILAN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ISTITUTO NANOSCIENZE (NANO-CNR)', 'CNR')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CONSIGLIO NAZIONALE DELLE RICERCHE (CNR)', 'CNR')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ISTITUTO NAZIONALE DI FISICA NUCLEARE', 'IST NAZL FIS NUCL')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('POLISH ACADEMY OF SCIENCES', 'POLISH ACAD SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CNRS - INSTITUTE OF PHYSICS (INP)', 'CNRS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('TECHNICAL UNIVERSITY OF MUNICH', 'TECH UNIVERSITY OF MUNCHEN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY COLLEGE LONDON', 'UCL')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF PARIS 11', 'UNIVERSITY OF PARIS SACLAY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST QUANTUM OPT', 'MAX PLANCK SOCIETY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('AUSTRIAN ACADEMY OF SCIENCES', 'AUSTRIAN ACAD SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS (CSIC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ETH ZURICH', 'ETH')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ECOLE POLYTECHNIQUE FEDERALE DE LAUSANNE', 'ECOLE POLYTECH FED LAUSANNE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE PARIS SACLAY (COMUE)', 'UNIVERSITY OF PARIS SACLAY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('LEIBNIZ UNIVERSITY OF HANNOVER', 'UNIVERSITY OF HANNOVER')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('COMMUNAUTE UNIVERSITY OFERSITE GRENOBLE ALPES', 'UNIVERSITY OF GRENOBLE ALPES')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('KARLSRUHE INSTITUTE OF TECHNOLOGY', 'KARLSRUHE INST TECHNOL')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('SCUOLA NORMALE SUPERIORE DI PISA', 'SCUOLA NORMALE SUPER PISA')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST PHYS KOMPLEXER SYST', 'MAX PLANCK SOCIETY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('RUPRECHT KARL UNIVERSITY HEIDELBERG', 'UNIVERSITY OF HEIDELBERG')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF LA SAPIENZA', 'SAPIENZA UNIVERSITY ROME')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('FORSCHUNGSZENTRUM JULICH', 'RESEARCH CENTER JULICH')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ROYAL INSTITUTE OF TECHNOLOGY', 'ROYAL INST TECHNOL')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE TOULOUSE III - PAUL SABATIER', 'UNIVERSITY OFERSITE DE TOULOUSE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CNRS - INSTITUTE OF CHEMISTRY (INC)', 'CNRS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('RHEIN WESTFAL TH AACHEN', 'RWTH AACHEN UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF GRENOBLE ALPES', 'UNIVERSITY OFERSITE GRENOBLE ALPES (UGA)')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('PALACKY UNIVERSITY OLOMOUC', 'PALACKY UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF PARIS SUD - PARIS XI', 'UNIVERSITY OF PARIS SACLAY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE DE TOULOUSE', 'UNIVERSITY OF TOULOUSE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('SORBONNE UNIVERSITY OFERSITE', 'SORBONNE UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('SORBONNE UNIVERSITY OF UPMC PARIS VI', 'SORBONNE UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('SORBONNE UNIVERSITY OFERSITES (COMUE)', 'SORBONNE UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UPMC SORBONNE UNIVERSITY', 'SORBONNE UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('HUNGARIAN ACADEMY OF SCIENCES', 'HUNGARIAN ACAD SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('HUMBOLDT UNIVERSITY OF BERLIN', 'HUMBOLDT UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE TOULOUSE III - PAUL SABATIER', 'UNIVERSITY OF TOULOUSE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('BARCELONA INSTITUTE OF SCIENCE & TECHNOLOGY', 'BARCELONA INST SCI & TECHNOL')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('PHYSIKALISCH-TECHNISCHE BUNDESANSTALT (PTB)', 'PHYS TECH BUNDESANSTALT')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF GRENOBLE 1', 'UNIVERSITY OF GRENOBLE ALPES')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE GRENOBLE ALPES (UGA)', 'UNIVERSITY OF GRENOBLE ALPES')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('TECHNISCHE UNIVERSITY OFERSITAT DRESDEN', 'DRESDEN UNIVERSITY OF TECHNOLOGY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CZECH ACADEMY OF SCIENCES', 'ACAD SCI CZECH REPUBLIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('COMMUNAUTE UNIVERSITY OFERSITE GRENOBLE ALPES', 'UNIVERSITY OF GRENOBLE ALPES')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ABDUS SALAM INTERNATIONAL CENTRE FOR THEORETICAL PHYSICS (ICTP)', 'ABDUS SALAAM INT CTR THEORET PHYS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF PARIS 11', 'UNIVERSITY OF PARIS SACLAY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('KATHOLIEKE UNIVERSITY OF LEUVEN', 'KU LEUVEN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('POLYTECHNIC UNIVERSITY OF MILAN', 'POLITECN MILAN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ISTITUTO NANOSCIENZE (NANO-CNR)', 'CNR')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CONSIGLIO NAZIONALE DELLE RICERCHE (CNR)', 'CNR')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CNRS - INSTITUTE OF PHYSICS (INP)', 'CNRS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF COMPLUTENSE MADRID', 'UNIVERSITY OF COMPLUTENSE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE SORBONNE PARIS CITE-USPC (COMUE)', 'SORBONNE UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST SOLID STATE RES', 'MAX PLANCK SOCIETY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('STFC RUTHERFORD APPLETON LABORATORY', 'RUTHERFORD APPLETON LAB')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF CLAUDE BERNARD - LYON 1', 'UNIVERSITY OF LYON')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF LYON 1', 'UNIVERSITY OF LYON')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CHALMERS UNIVERSITY OF TECHNOLOGY', 'CHALMERS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('FRIEDRICH SCHILLER UNIVERSITY OF JENA', 'UNIVERSITY OF JENA')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('LUDWIG MAXIMILIANS UNIVERSITY OF MUNCHEN', 'UNIVERSITY OF MUNICH')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF PARIS 11', 'UNIVERSITY OF PARIS SACLAY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST FESTKORPERFORSCH', 'MAX PLANCK SOCIETY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF BORDEAUX 1', 'UNIVERSITY OF BORDEAUX')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE DE BORDEAUX', 'UNIVERSITY OF BORDEAUX')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE FEDERALE TOULOUSE MIDI-PYRENEES (COMUE)', 'UNIVERSITY OF TOULOUSE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE DE TOULOUSE', 'UNIVERSITY OF TOULOUSE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE DE STRASBOURG', 'UNIVERSITY OF STRASBOURG')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITES DE STRASBOURG ETABLISSEMENTS ASSOCIES', 'UNIVERSITY OF STRASBOURG')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE DE STRASBOURG', 'UNIVERSITY OF STRASBOURG')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('BASQUE FOUNDATION FOR SCIENCE', 'BASQUE FDN SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSIDADE DE LISBOA', 'UNIVERSITY OF LISBON')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST QUANTUM OPT', 'MAX PLANCK SOCIETY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST SCI LIGHT', 'MAX PLANCK SOCIETY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('INSTITUTE OF PHYSICS - POLISH ACADEMY OF SCIENCES', 'POLISH ACAD SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('POLISH ACADEMY OF SCIENCES', 'POLISH ACAD SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('DORTMUND UNIVERSITY OF TECHNOLOGY', 'UNIVERSITY OF DORTMUND')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('INSTITUT DE CIENCIES FOTONIQUES (ICFO)', 'ICFO')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ICFO INST CIENCIES FOTON', 'ICFO')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('EBERHARD KARLS UNIVERSITY OF TUBINGEN', 'UNIVERSITY OF TUBINGEN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('POLYTECHNIC UNIVERSITY OF TURIN', 'POLITECN TURIN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CEA SACLAY', 'CEA')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('SLOVAK ACADEMY OF SCIENCES', 'SLOVAK ACAD SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('KARLSRUHE INSTITUTE OF TECHNOLOGY', 'UNIVERSITY OF KARLSRUHE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('PAUL SCHERRER INSTITUTE', 'PAUL SCHERRER INST') 
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ISTITUTO NANOSCIENZE (NANO-CNR)', 'CNR')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF PARIS SUD - PARIS XI', 'UNIVERSITY OF PARIS SACLAY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF GRENOBLE 1', 'UNIVERSITY OF GRENOBLE ALPES')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('POLYTECHNIC UNIVERSITY OF MILAN', 'POLITECN MILAN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF LONDON IMPERIAL COLL SCI TECHNOL & MEDICAL', 'IMPERIAL COLLEGE LONDON')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ISTITUTO NAZIONALE DI OTTICA (INO-CNR)', 'CNR')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('INSTITUTE OF PHYSICS OF THE CZECH ACADEMY OF SCIENCES', 'ACAD SCI CZECH REPUBLIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIA DE MATERIALES DE BARCELONA (ICMAB)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIA DE MATERIALES DE MADRID (ICMM)', 'CISC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO ANDALUZ DE BIOLOGIA MOLECULAR Y MEDICINA REGENERATIVA (CABIMER)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE ASTROBIOLOGIA (INTA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE BIOLOGIA MOLECULAR SEVERO OCHOA (CBM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE CIENCIAS MEDIOAMBIENTALES (CCMA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE EDAFOLOGIA Y BIOLOGIA APLICADA DEL SEGURA (CEBAS)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE FISICA MIGUEL A. CATALAN (CFMAC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE INVESTIGACION EN NANOMATERIALES Y NANOTECNOLOGIA (CINN)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE INVESTIGACION Y DESARROLLO PASCUAL VILA (CID-CSIC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE INVESTIGACIONES BIOLOGICAS (CIB)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO DE QUIMICA Y MATERIALES DE ARAGON (CEQMA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO MEDITERRANEO DE INVESTIGACIONES MARINAS Y AMBIENTALES (CMIMA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO NACIONAL DE ACELERADORES (CNA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO NACIONAL DE BIOTECNOLOGIA (CNB)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - CENTRO NACIONAL DE MICROELECTRONICA (CNM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - ESTACION EXPERIMENTAL DE AULA DEI (EEAD)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - ESTACION EXPERIMENTAL DE ZONAS ARIDAS (EEZA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - ESTACION EXPERIMENTAL DEL ZAIDIN (EEZ)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - GEOCIENCIAS BARCELONA (GEO3BCN)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO ANDALUZ DE CIENCIAS DE LA TIERRA (IACT)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE ACUSTICA (IA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE AGRICULTURA SOSTENIBLE (IAS)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE ASTROFISICA DE ANDALUCIA (IAA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE BIOMEDICINA Y BIOTECNOLOGIA DE CANTABRIA (IBBTEC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE BIOQUIMICA VEGETAL Y FOTOSINTESIS (IBVF)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CARBOQUIMICA (ICB)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CATALISIS Y PETROLEOQUIMICA (ICP)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CERAMICA Y VIDRIO (ICV)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIA & TECNOLOGIA DE POLIMEROS (ICTP)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIAS AGRARIAS (ICA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIAS DEL ESPACIO (ICE)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIAS DEL MAR (ICM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIAS MARINAS DE ANDALUCIA (ICMAN)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE CIENCIAS MATEMATICAS (ICMAT)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE ESTRUCTURA DE LA MATERIA (IEM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE FISICA APLICADA (IFA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE FISICA CORPUSCULAR (IFIC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE FISICA DE CANTABRIA (IFCA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE FISICA FUNDAMENTAL (IFF)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE INVESTIGACION EN RECURSOS CINEGETICOS (IREC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE INVESTIGACIONES BIOMEDICAS ALBERTO SOLS (IIBM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE INVESTIGACIONES MARINAS (IIM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE MATEMATICAS FISICA FUNDAMENTAL (IMAFF)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE MICROELECTRONICA DE BARCELONA (IMB-CNM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE MICROELECTRONICA DE MADRID (IMM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE MICROELECTRONICA DE SEVILLA (IMS-CNM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE OPTICA (DAZA DE VALDES)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE PRODUCTOS LACTEOS DE ASTURIAS (IPLA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE PRODUCTOS NATURALES Y AGROBIOLOGIA (IPNA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE QUIMICA AVANZADA DE CATALUNA (IQAC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE QUIMICA FISICA ROCASOLANO (IQFR)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE QUIMICA MEDICA (IQM)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE QUIMICA ORGANICA GENERAL (IQOG)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE RECURSOS NATURALES Y AGROBIOLOGIA DE SALAMANCA (IRNASA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO DE TECNOLOGIAS FISICAS Y DE LA INFORMACION (ITEFI)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - INSTITUTO MEDITERRANEO DE ESTUDIOS AVANZADOS (IMEDEA)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('INFN', 'IST NAZL FIS NUCL')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('NATIONAL PHYSICAL LABORATORY - UK', 'NATL PHYS LAB')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CHALMERS UNIVERSITY OF TECHNOL', 'CHALMERS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('POLITECN TURIN', 'POLITECN TORINO')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CSIC - UAM - INSTITUT DE FISICA TEORICA (IFT)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('TECHNISCHE UNIVERSITY OFERSITAT WIEN', 'TU WIEN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CENTER FOR THEORETICAL PHYSICS - POLISH ACADEMY OF SCIENCES', 'POLISH ACAD SCI')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST KERNPHYS', 'MAX PLANCK SOCIETY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITAT SIEGEN', 'UNIVERSITY OF SIEGEN')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CTR UNIVERSITY OF PARIS SUD', 'UNIVERSITY OF PARIS SACLAY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('WROCLAW UNIVERSITY OF SCIENCE & TECHNOLOGY', 'WARSAW UNIVERSITY OF TECHNOL')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('CZECH TECHNICAL UNIVERSITY PRAGUE', 'CZECH TECH UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('JOHANNES KEPLER UNIVERSITY OF LINZ', 'JOHANNES KEPLER UNIVERSITY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('PAUL DRUDE INSTITUTE FOR SOLID STATE ELECTRONICS', 'PAUL DRUDE INST FESTKORPERELEKT')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE LIBRE DE BRUXELLES', 'FREE UNIVERSITY OF BRUSSELS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE DE LILLE', 'UNIVERSITY OF LILLE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF LILLE 1', 'UNIVERSITY OF LILLE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('DEUTSCHES ELEKTRONEN-SYNCHROTRON (DESY)', 'DESY')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('HEINRICH HEINE UNIVERSITY DUSSELDORF', 'UNIVERSITY OF DUSSELDORF')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSIDADE DE COIMBRA', 'UNIVERSITY OF COIMBRA')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ECOLE NORMALE SUPERIEURE (ENS)', 'ENS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ECOLE NORMALE SUPERIEURE DE PARIS', 'ENS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('JOZEF STEFAN INSTITUTE', 'JOZEF STEFAN INST')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('ECHNISCHE UNIVERSITY OFERSITAT CHEMNITZ', 'TECH UNIVERSITY OF CHEMNITZ')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITE DE LORRAINE', 'UNIVERSITY OF LORRAINE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('INSTITUTO DE CIENCIA DE MATERIALES DE SEVILLA (ICMS-CSIC)', 'CSIC')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('MAX PLANCK INST MATH SCI', 'MAX PLANCK SOCIETY')   
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSITA DI MODENA E REGGIO EMILIA', 'UNIVERSITY OF MODENA & REGGIO EMILIA')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OF FLORENCE', 'UNIVERSITY OF FIRENZE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('VRIJE UNIVERSITY OFERSITEIT BRUSSEL', 'FREE UNIVERSITY OF BRUSSELS')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('HELMHOLTZ-ZENTRUM FUER MATERIALIEN UND ENERGIE GMBH (HZB)', 'HELMHOLTZ ZENTRUM BERLIN MAT & ENERGIE')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('HEWLETT PACKARD LABS', 'HEWLETT PACKARD')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('VRIJE UNIVERSITY OFERSITEIT AMSTERDAM', 'VRIJE UNIVERSITY OF AMSTERDAM')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('UNIVERSITY OFERSIDADE DO PORTO', 'UNIVERSITY OF PORTO')
inst['final_standardized_organization'] = inst['final_standardized_organization'].replace('HELMHOLTZ ZENTRUM BERLIN MAT & ENERGIE', 'HELMHOLTZ ASSOCIATION')

# Check the changes
inst[['organization', 'standardized_organization','final_standardized_organization']].head()

,organization,standardized_organization,final_standardized_organization
0,UNIV PATRAS,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS
1,UNIV PATRAS,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS
2,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS
3,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS,UNIVERSITY OF PATRAS
4,UNIV DURHAM,UNIVERSITY OF DURHAM,UNIVERSITY OF DURHAM


In [10]:
inst.final_standardized_organization.value_counts()

CNRS                                   34699
UNIVERSITY OF PARIS SACLAY             14949
CNR                                    11100
UNIVERSITY OF GRENOBLE ALPES            9582
MAX PLANCK SOCIETY                      8180
                                       ...  
UNIVERSITY OF & POLYTECH HOSP LA FE        1
SAKHAROV INT STATE ECOL UNIVERSITY         1
QUANTUMWISE AS                             1
CTR BIO SYST GENOM                         1
ARQUIMEA RES CTR                           1
Name: final_standardized_organization, Length: 9172, dtype: int64

In [11]:
# inst.to_csv('../05_허지수박사/quant_inst_ed_eu_fixed.csv', encoding='ISO-8859-1')
inst.to_csv('QuanTech_R/R file/quant_inst_ed_eu_fixed.csv', encoding='ISO-8859-1', index=False)